In [1]:
import pandas as pd
import numpy as np
from functions import *
from pytorch_util import *
from pytorch_models import ConvBatchLeaky1D,GRU_NCL
from torch.nn import Linear,Sequential,LeakyReLU,Dropout,L1Loss
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
train = pd.read_csv("../Data/train.csv", 
                    dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32})
train = train.values
train[:,0] = normalize(train[:,0])

In [3]:
## change ##
batch_size = 16
clip = .5

Prepare data

In [40]:
train_gen = mixupWrapper(SequenceGen(train),1e-2)
train_gen = DataLoader(train_gen,batch_size,False,num_workers=2)

val_gen = SequenceGen(train,False)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=2)

Build model

In [41]:
convNets = Sequential(ConvBatchLeaky1D(1,2,5,stride=2),
                      ConvBatchLeaky1D(2,4,5,stride=2),
                      ConvBatchLeaky1D(4,8,5,stride=2),
                      ConvBatchLeaky1D(8,16,5,stride=2),
                      ConvBatchLeaky1D(16,32,5,stride=2),
                      DenseBlock(32),
                      DenseBlock(64),
                      Dropout(0.5),
                      DenseBlock(128),
                      Dropout(0.5),
                      DenseBlock(256),
                      Dropout(0.5),
                      GRU_NCL(512,512,batch_first=True,bidirectional=True,returnH=True))

In [42]:
linear = Sequential(Dropout(0.5),Linear(512*2,128),LeakyReLU(0.1,True),Dropout(0.25),Linear(128,1))

In [43]:
model = CNN_RNN2seq(convNets,linear).to('cuda:0')

In [44]:
opt = Adam(trainable_parameter(model),lr=1e-3)

In [45]:
loss_func_nor = loss_func_generator(L1Loss())

Training

In [46]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:3.1221826328681064, val_loss:3.311523675918579
epoch:1, train_loss:3.025606407569005, val_loss:3.1750078201293945
epoch:2, train_loss:2.8943232968449593, val_loss:3.1384897232055664
epoch:3, train_loss:2.76500434256517, val_loss:3.0110068321228027
epoch:4, train_loss:2.69107985496521, val_loss:2.890018939971924
Training completed in 70.78618311882019s


In [47]:
model = fit(5, model, loss_func_nor, opt, train_gen, val_gen,clip=clip)

epoch:0, train_loss:2.6779806327361326, val_loss:2.9438977241516113
epoch:1, train_loss:2.54689212086109, val_loss:3.252962350845337
epoch:2, train_loss:2.498875929759099, val_loss:3.031766653060913
epoch:3, train_loss:2.467815732726684, val_loss:2.9198756217956543
epoch:4, train_loss:2.4569153986298122, val_loss:2.958057403564453
Training completed in 71.09829664230347s


Submission

In [102]:
submission = pd.read_csv('../Data/sample_submission.csv',)

In [145]:
test_gen = SequenceGenTest(submission.seg_id.tolist())
test_gen = DataLoader(test_gen,batch_size,False,num_workers=2)

In [155]:
submission.iloc[:,1] = predict(model,test_gen)

In [164]:
submission.to_csv('../Submission/sub1.csv',index=False)